In [ ]:
from flask import Flask,render_template,request,redirect,url_for,session
from flask_pymongo import PyMongo
import os

from flask import flash

# from decorators import login_required
from functools import wraps

app = Flask(__name__)
app.config['SECRET_KEY'] = os.urandom(24)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

def login_required(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print('session get account: ',session.get('account'))
        if session.get('account'):
            return func(*args,**kwargs)
        else:
            return redirect(url_for('login'))
        
    return wrapper

@app.route('/')
def index():
    hotel_name=[]
    hotel_url=[]
    hotel_id = []

    with open('city_hotelname_hid_hurl','r',encoding='utf-8') as f:
        for line in f:
            a = line.split()
            
            hotel_name.append(''.join(a[1:len(a)-2]))
            hotel_url.append(a[len(a)-1])
            hotel_id.append(a[len(a)-2])
    return render_template('index.html',hotel_name=hotel_name,hotel_url=hotel_url)

@app.route('/login/',methods=['GET','POST'])
def login():
    if request.method == 'GET':
        return render_template('login.html')
    else:
        account = request.form.get('account')
        password = request.form.get('password')
        user_collection=mongo.db.hotel
        user = user_collection.find_one({"$and":[ {'name':account}
                                             ,{'password':password}
                                             ]})#查無回傳None
        if user:
            session['account'] = user['name']
            session.permanent = True #如果想在31天都不需要登入
            
            return redirect(url_for('index'))
        else:
            return '名稱或密碼錯誤'


@app.route('/register/',methods=['GET','POST'])
def register():
    if request.method == 'GET':
        return render_template('register.html')
    
    
    else:
        account = request.form.get('account')
        password1 = request.form.get('password1')
        password2 = request.form.get('password2')
        
        user_collection=mongo.db.hotel
        if user_collection.find_one({'name':account}):#查無回傳None
            return '該名稱已被註冊，請更換'
        else:
            if password1!=password2:
                return '兩次密碼不相同，請核對後再填寫'
            else:
                user_collection.insert_one({'name':account,\
                            'password':password1,\
                            'public_key':'',\
                            'private_key':'',\
                            'ratings':[]
                        })
                return redirect(url_for('login'))
        

@app.route('/addhotel/',methods=['GET','POST'])
@login_required
def addhotel():
    if request.method =='GET':
        return render_template('addhotel.html')
    else:
        hotelname = request.form.get('hotelname')
        address = request.form.get('address')
        introduction = request.form.get('introduction')
        landlord = session.get('account')#房東
        
        rent_hotel_collection=mongo.db.rent_hotel
        rent_hotel_collection.insert_one({'landlord':landlord,\
                            'hotelname':hotelname,\
                            'address':address,\
                            'introduction':introduction,\
                            'ratings':[]
                        })
        return redirect(url_for('index'))


    
    
##################
#################
@app.route('/test')
def test():
    hotel_name=[]
    hotel_url=[]
    hotel_id = []

    with open('city_hotelname_hid_hurl','r',encoding='utf-8') as f:
        for line in f:
            a = line.split()
            
            hotel_name.append(''.join(a[1:len(a)-2]))
            hotel_url.append(a[len(a)-1])
            hotel_id.append(a[len(a)-2])
    user_collection=mongo.db.rent_hotel
    context = {
        'rent_hotels':[i for i in user_collection.find().sort('_id',-1)]
    }
    return render_template('newhotel.html',hotel_name=hotel_name,hotel_url=hotel_url,**context)
    
    
###############
###############
@app.route('/sendmoney/<renthotel_id>/',methods=['GET','POST'])
# @login_required
def sendmoney(renthotel_id):
    if request.method =='GET':
        return render_template('sendmoney.html')
    else:
        hotelname = request.form.get('hotelname')
        address = request.form.get('address')
        introduction = request.form.get('introduction')
        landlord = session.get('account')#房東
        
        rent_hotel_collection=mongo.db.rent_hotel
        rent_hotel_collection.insert_one({'landlord':landlord,\
                            'hotelname':hotelname,\
                            'address':address,\
                            'introduction':introduction,\
                            'ratings':[]
                        })
        return redirect(url_for('index'))
    


if __name__=='__main__':    
    app.run(debug=True,use_reloader=False) 

In [99]:

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import  MatrixFactorizationModel
  
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

user_collection=mongo.db.user#評價資料giveid寫入資料庫    

def CreateSparkContext():
    sparkConf = SparkConf()                                                       \
                         .setAppName("Recommend")                         \
                         .set("spark.ui.showConsoleProgress", "false") \
               
    sc = SparkContext(conf = sparkConf)
    print("master="+sc.master)
    SetLogger(sc)
    SetPath(sc)
    return (sc)

def SetPath(sc):
    global Path
    if sc.master[0:5]=="local" :
        Path="file:/home/hduser/pythonwork/ipynotebook/"
    else:   
        Path="hdfs://master:9000/user/hduser/"

def SetLogger( sc ):
    logger = sc._jvm.org.apache.log4j
    logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
    logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )
    logger.LogManager.getRootLogger().setLevel(logger.Level.ERROR)
    
def PrepareData(sc): 
    print("開始讀取旅館ID與名稱對照表...")
    itemRDD = sc.textFile(Path+"data/givehotel") 
    movieTitle= itemRDD.map( lambda line : line.split())     \
                                   .map(lambda a: (float(a[0]),a[2]+' '+a[1]))       \
                                   .collectAsMap()       
    print(movieTitle[1])
    return(movieTitle)

def RecommendMovies(model, movieTitle, inputUserID): 
    RecommendMovie = model.recommendProducts(inputUserID, 10) 
    print("針對使用者id" + str(inputUserID) + "推薦下列電影:")
    for rmd in RecommendMovie:
        print(rmd)
        print  ("針對使用者id {0} 推薦旅館 {1} 推薦評分 {2}". \
            format( rmd[0],movieTitle[rmd[1]],rmd[2]))
        print(type(rmd[1]),rmd[1])
    for i in range(2,101):
        RecommendMovie = model.recommendProducts(i, 10) 
        for rmd in RecommendMovie:
            user_collection.update_one({'uid':rmd[0]},\
                                        {'$push':{'rcmhotel':{str(rmd[1]):movieTitle[rmd[1]]}}})
        

# def RecommendUsers(model, movieTitle, inputMovieID) :
#     RecommendUser = model.recommendUsers(inputMovieID, 10) 
#     print( "針對電影 id {0} 電影名:{1}推薦下列使用者id:". \
#            format( inputMovieID,movieTitle[inputMovieID]))
#     for rmd in RecommendUser:
#         print  ("針對使用者id {0}  推薦評分 {1}".format( rmd[0],rmd[2]))


def loadModel(sc):
    try:        
        model = MatrixFactorizationModel.load(sc, Path+"ALSmodel")
        print("載入ALSModel模型")
    except Exception:
        print( "找不到ALSModel模型,請先訓練")
    return model 



def Recommend(model):
        RecommendMovies(model, movieTitle,1)
#     if sys.argv[1]=="--U":
#         RecommendMovies(model, movieTitle,int(sys.argv[2]))
#     if sys.argv[1]=="--M": 
#         RecommendUsers(model, movieTitle,int(sys.argv[2]))


if __name__ == "__main__":
#     sc=CreateSparkContext()
    SetPath(sc)
    print("==========資料準備===============")
    movieTitle= PrepareData(sc)
    print("==========載入模型===============")
    model=loadModel(sc)
    print("==========進行推薦===============")
    Recommend(model)


==========資料準備===============
開始讀取旅館ID與名稱對照表...
新驛旅店-復興北路店 台灣中山區
==========載入模型===============
載入ALSModel模型
==========進行推薦===============
針對使用者id1推薦下列電影:
Rating(user=1, product=1984, rating=9.918401473946252)
針對使用者id 1 推薦旅館 異宿風巢景觀旅店 台灣安平 推薦評分 9.918401473946252
<class 'int'> 1984
Rating(user=1, product=5914, rating=9.51288825341953)
針對使用者id 1 推薦旅館 長成民宿 台灣澎湖 推薦評分 9.51288825341953
<class 'int'> 5914
Rating(user=1, product=5233, rating=8.991700080150022)
針對使用者id 1 推薦旅館 花蓮莊家堡田園民宿 台灣吉安 推薦評分 8.991700080150022
<class 'int'> 5233
Rating(user=1, product=3394, rating=8.984940457954504)
針對使用者id 1 推薦旅館 綠原度假旅社 台灣雲林 推薦評分 8.984940457954504
<class 'int'> 3394
Rating(user=1, product=5042, rating=8.942164893619207)
針對使用者id 1 推薦旅館 逸心園民宿 台灣壽豐 推薦評分 8.942164893619207
<class 'int'> 5042
Rating(user=1, product=1800, rating=8.75353940347434)
針對使用者id 1 推薦旅館 旅人船說背包聚落 台灣中西區 推薦評分 8.75353940347434
<class 'int'> 1800
Rating(user=1, product=2353, rating=8.548618532338239)
針對使用者id 1 推薦旅館 旅捷商務旅館 台灣苓雅 推薦評分 8.548618532338

### 寫入資料庫

In [22]:
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

user_collection=mongo.db.rating #評價資料giveid寫入資料庫
j = 0 
with open('data/giveid','r',encoding='utf-8') as f:
    for i in f:
        if j==100:
            break
        else:
            a = i.strip().split(' ')#userid:a[0] hotelid:a[1] rating:a[2]
            user_collection.insert_one({'userid':a[0],'hotelid':a[1],'rating':a[2]})
            j+=1

In [39]:
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

user_collection=mongo.db.hotel_id #旅館對應id... giveid寫入資料庫
j = 0 
with open('data/givehotel','r',encoding='utf-8') as f:
    for i in f:
        a = i.strip().split(' ')#hotelid:a[0],location:a[1],hotelname:a[2],url:a[4]
        user_collection.insert_one({'hotelid':a[0],\
                                  'location':a[1],\
                                  'hotelname':a[2],\
                                  'orginalid':a[3],\
                                  'url':a[4]})
#         print(user_collection.estimated_document_count())
        


In [37]:
user_collection.estimated_document_count()

6202

In [96]:
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

user_collection=mongo.db.user
# user_collection.update_many({},{'$set':{'rcmhotel':[]}})#新增陣列欄位********
# user_collection.delete_many({'rcmhoteml':[]})#刪除object
# user_collection.update_many({},{'$set':{'eth':100}})#新增eth欄位*******
#user_collection.update_many({},{'$rename':{'ett':'eth'}})＃更改document 名稱
# user_collection.update_many({},{'$pull':{'eth':'eth'}})＃更改document 名稱
# user_collection.update_many({},{'$pull':{}})#對陣列刪除
# user_collection.update_many({},{'$unset':{'eth':100}})#刪除欄位




#建立user collection
#**************
# j = 0
# with open('data/hid-uid-rat.json','r',encoding='utf-8') as f:
#     data = json.load(f)
#     print(type(data))
#     for i,k in zip(data.keys(),data.values()):
#         if j == 100:
#             break
#         else:
#             j+=1
            
#             user_collection.insert_one({'account':i,\
#                             'uid':j,\
#                             'password':'',\
#                             'public_key':'',\
#                             'private_key':'',\
#                             'ratings':k
#                         })

In [69]:
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

user_collection=mongo.db.transaction #
user_collection.insert_one({})


In [86]:
dir(user_collection)

['_BaseObject__codec_options',
 '_BaseObject__read_concern',
 '_BaseObject__read_preference',
 '_BaseObject__write_concern',
 '_Collection__create',
 '_Collection__create_index',
 '_Collection__database',
 '_Collection__find_and_modify',
 '_Collection__full_name',
 '_Collection__name',
 '_Collection__write_response_codec_options',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_aggregate',
 '_aggregate_one_result',
 '_command',
 '_count',
 '_delete',
 '_delete_retryable',
 '_insert',
 '_insert_one',
 '_legacy_write',
 '_map_reduce',
 '_read_preference_for',
 '_socket_for_reads',
 '_socket_for_writ

# 加入hotel_img hotel_rat

In [4]:
from flask import Flask
from flask_pymongo import PyMongo
import json

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

hotel_collection=mongo.db.hotel_id

# user_collection.update_many({},{'$set':{'rcmhotel':[]}})#新增陣列欄位********
# user_collection.delete_many({'rcmhoteml':[]})#刪除object
# user_collection.update_many({},{'$set':{'eth':100}})#新增eth欄位*******
#user_collection.update_many({},{'$rename':{'ett':'eth'}})＃更改document 名稱
# user_collection.update_many({},{'$pull':{'eth':'eth'}})＃更改document 名稱
# user_collection.update_many({},{'$pull':{}})#對陣列刪除
# user_collection.update_many({},{'$unset':{'eth':100}})#刪除欄位

In [3]:
hotel_collection.update_many({'uid':2},{'$set':{'test':100}})#新增eth欄位*******

In [20]:
with open('hotel_img','r',encoding='utf-8') as f:
    for i in f:
        a = i.split()
        if len(a) != 2:
            continue
        else:
            hotel_collection.update_one({'hotelname':a[0]},{'$set':{'imgpath':a[1]}})
        


## 更改資料庫中hotel_rating型態 string to int

In [21]:
with open('hotel_rat','r',encoding='utf-8') as f:
    for i in f:
        a = i.split()
        if len(a) != 2:
            continue
        else:
            hotel_collection.update_one({'hotelname':a[0]},{'$set':{'hotel_rating':a[1]}})
        


In [81]:
# for i in hotel_collection.find({}):
#     hotel_collection.update_one({'hotelid':i.get('hotelid')},{'$set':{'hotelid':int(i.get('hotelid'))}})
    
a = hotel_collection.find({'hotelid':{'$lt':10}}).limit(10)
for i in a :
    print(i)

{'_id': ObjectId('5da2b878d686edf8ccf3c6eb'), 'hotelid': 1, 'location': '台灣中山區', 'hotelname': '新驛旅店-復興北路店', 'orginalid': '8263705', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d8263705-Reviews-Cityinn_Hotel_Plus_Fuxing_N_Rd_Branch-Zhongshan_District_Taipei.html', 'imgpath': 'https://media-cdn.tripadvisor.com/media/photo-w/0f/5a/67/42/caption.jpg', 'hotel_rating': '5'}
{'_id': ObjectId('5da2b878d686edf8ccf3c6ec'), 'hotelid': 2, 'location': '台灣中山區', 'hotelname': '優美飯店', 'orginalid': '1372927', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d1372927-Reviews-YoMi_Hotel-Zhongshan_District_Taipei.html', 'imgpath': 'https://media-cdn.tripadvisor.com/media/photo-w/17/0a/d1/88/caption.jpg', 'hotel_rating': '4.5'}
{'_id': ObjectId('5da2b878d686edf8ccf3c6ed'), 'hotelid': 3, 'location': '台灣中山區', 'hotelname': '圓山大飯店', 'orginalid': '306446', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d306446-Reviews-Grand_Hotel_Taipei-Zhongshan_District_Taipei.html